<a href="https://colab.research.google.com/github/Zuyuf/Machine_Learning_Specialization/blob/main/1-Machine_Learning_Foundation/5%20-%20WEEK%205%20-%20Recommending%20Products/2_WEEK_5_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install TuriCreate and Connect to Drive**

In [1]:
!pip install turicreate

     |████████████████████████████████| 92.0MB 102kB/s 
     |████████████████████████████████| 86.4MB 48kB/s 
     |████████████████████████████████| 3.4MB 43.6MB/s 
     |████████████████████████████████| 327kB 47.6MB/s 
     |████████████████████████████████| 450kB 41.2MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 3.8MB 37.9MB/s 
  Created wheel for resampy: filename=resampy-0.2.1-cp36-none-any.whl size=320848 sha256=b8ec6b6829f4ff64d1ce676a5ee24f590b405ef5be35bac4ed98b9c64447bdf0
  Stored in directory: /root/.cache/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=d7ed420b30bc9f102626d03bd25b938df8b8d85b67736082ecb00f52b15b3129
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built resampy gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2,

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import turicreate

# Load some music data

In [8]:
song_data = turicreate.SFrame('/content/gdrive/My Drive/Turicreate/Week 5/song_data.sframe/')
song_data.column_names

<bound method SFrame.column_names of Columns:
	user_id	str
	song_id	str
	listen_count	int
	title	str
	artist	str
	song	str

Rows: 1116609

Data:
+-------------------------------+--------------------+--------------+
|            user_id            |      song_id       | listen_count |
+-------------------------------+--------------------+--------------+
| b80344d063b5ccb3212f76538f... | SOAKIMP12A8C130995 |      1       |
| b80344d063b5ccb3212f76538f... | SOBBMDR12A8C13253B |      2       |
| b80344d063b5ccb3212f76538f... | SOBXHDL12A81C204C0 |      1       |
| b80344d063b5ccb3212f76538f... | SOBYHAJ12A6701BF1D |      1       |
| b80344d063b5ccb3212f76538f... | SODACBL12A8C13C273 |      1       |
| b80344d063b5ccb3212f76538f... | SODDNQT12A6D4F5F7E |      5       |
| b80344d063b5ccb3212f76538f... | SODXRTY12AB0180F3B |      1       |
| b80344d063b5ccb3212f76538f... | SOFGUAY12AB017B0A8 |      1       |
| b80344d063b5ccb3212f76538f... | SOFRQTD12A81C233C0 |      1       |
| b80344d063b5c

# Compute the number of unique users for each of these artists: 'Kanye West', 'Foo Fighters', 'Taylor Swift' and 'Lady GaGa'.

###select Enytries Artist == 'Kanye West' >> get the Unique entries of 'user_id'

In [11]:
kanye_listners = song_data[ song_data['artist'] == 'Kanye West' ]['user_id'].unique()
len(kanye_listners)

2522

In [12]:
foo_listners = song_data[ song_data['artist'] == 'Foo Fighters' ]['user_id'].unique()
len(foo_listners)

2055

In [13]:
taylor_listners = song_data[ song_data['artist'] == 'Taylor Swift' ]['user_id'].unique()
len(taylor_listners)

3246

In [14]:
lady_listners = song_data[ song_data['artist'] == 'Lady GaGa' ]['user_id'].unique()
len(lady_listners)

2928

# Using groupby-aggregate to find the most popular and least popular artist:

In [25]:
artist_popularity = song_data.groupby(key_column_names='artist', operations={'total_count': turicreate.aggregate.SUM('listen_count')})
artist_popularity = artist_popularity.sort(['total_count'], ascending=False)

**Most Popular Artist is**

In [23]:
artist_popularity.head(1)['artist'][0]

'Kings Of Leon'

**Least Popular Artist is**

In [24]:
artist_popularity.tail(1)['artist'][0]

'William Tabbert'

# Using groupby-aggregate to find the most recommended songs:

### Split the data into 80% training, 20% testing, using seed=0, as was done in the Jupyter notebook above.

In [38]:
train_data,test_data = song_data.random_split(.8,seed=0)

### Train an item_similarity_recommender, using the training data.

In [39]:
personalized_model = turicreate.item_similarity_recommender.create(train_data,
                                                                  user_id = 'user_id',
                                                                  item_id = 'song')

Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a rating column, specify the column name to be used as target

and use a method that allows the use of a target.

Preparing data set.

Data has 893580 observations with 66085 users and 9952 items.

...

| 2.01s                               | 100              | 9952            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 2.10279s

### Next, we are going to make recommendations for the users in the test data

but there are over 200,000 users (58,628 unique users) in the test set. Computing recommendations for these many users can be slow in some computers. Thus, we will use only the first 10,000 users only in this question. Using this command to select this subset of users:

In [40]:
subset_test_users = test_data['user_id'].unique()[0:10000]

### Let’s compute one recommended song for each of these test users. Use this command to compute these recommendations:

In [41]:
subset_test_users_recommendation = personalized_model.recommend(subset_test_users,k=1)

recommendations finished on 1000/10000 queries. users per second: 3459.75

recommendations finished on 2000/10000 queries. users per second: 3471.99

recommendations finished on 3000/10000 queries. users per second: 3396.52

recommendations finished on 4000/10000 queries. users per second: 3410.6

recommendations finished on 5000/10000 queries. users per second: 3418.53

recommendations finished on 6000/10000 queries. users per second: 3315.35

recommendations finished on 7000/10000 queries. users per second: 3329.17

recommendations finished on 8000/10000 queries. users per second: 3343.45

recommendations finished on 9000/10000 queries. users per second: 3334.9

recommendations finished on 10000/10000 queries. users per second: 3333.62

### Finally, we can use .groupby() to find the most recommended song!

In [42]:
subset_test_users_recommendation = subset_test_users_recommendation.groupby(key_column_names='song', 
                                                                            operations={'count': turicreate.aggregate.COUNT()} )
subset_test_users_recommendation.sort(['count'], ascending=False)

song,count
Undo - Björk,426
Secrets - OneRepublic,398
Revelry - Kings Of Leon,230
You're The One - DwightYoakam ...,154
Hey_ Soul Sister - Train,114
Sehr kosmisch - Harmonia,112
Fireflies - CharttraxxKaraoke ...,104
Horn Concerto No. 4 in Eflat K495: II. Romance ...,89
OMG - Usher featuringwill.i.am ...,69
The Scientist - Coldplay,50
